# Transductive Transfer Learning Setting

### Libraries

In [ ]:
## libs 
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

## Keras
from keras.layers import Lambda, Input, Dense, Conv2D, Conv2DTranspose, Flatten, Reshape
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

## Basic
from tqdm import tqdm_notebook as tqdm
import argparse
import os
import random
import itertools
import time

# Computation
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.preprocessing import MinMaxScaler

import scipy
from scipy.stats.stats import pearsonr 

## Visualization
import matplotlib.pyplot as plt
import seaborn as sns

## Network Processing
import networkx as nx
from networkx.generators import random_graphs

## node colour
color_map = ["steelblue"]

### Supporting Functions

In [ ]:
## supporting functions
from support.preprocessing import sort_adjacency, reshape_A, calculate_A_shape, reconstruct_adjacency, pad_matrix, unpad_matrix, prepare_in_out
from support.metrics import compute_mig, compute_mi
from support.graph_generating import generate_single, generate_manifold, generate_topol_manifold, generate_topol_manifold
from support.latent_space import vis2D, visDistr
from support.comparing import compare_manifold_adjacency, compare_topol_manifold


## graph sampling
from sampling import ForestFire, Metropolis_Hastings, Random_Walk, Snowball, Ties, Base_Samplers

## Probabilistic Graph Models

### Generate Graph Instance

In [ ]:
def get_graph(dataArgs, trainArgs): 
    
    ## Generate Graph Type ______________________________________________
    
    graph_type = random.choice(trainArgs["input_networks"])
    params = np.zeros((trainArgs["n_params"]))

    if dataArgs["fix_n"] == True:
        n = dataArgs["n_max"] # generate fixed number of nodes n_max
    else:
        n = random.randint(2, dataArgs["n_max"]) # generate number of nodes n between 1 and n_max and
            
    if graph_type == "ER":
        
        p = np.random.rand(1)  # float in range 0 - 1 
        g = random_graphs.erdos_renyi_graph(n, p, seed=None, directed=False)
        
        params[0] = 1
        params[1] = p
        
        
    if graph_type == "BA":
        
        e = np.clip(random.randint(1, max(1, n - 1)), 1, n-1) # generate number of nodes n between 1 and n_max and
        g = random_graphs.barabasi_albert_graph(n, e, seed=None)
        
        params[2] = 1
        params[3] = e
        
        
    if graph_type == "SW":
        
        k = random.randint(0, n - 1)
        p = np.random.rand(1)  # float in range 0 - 1 
        g = random_graphs.newman_watts_strogatz_graph(n, k, p, seed=None) # no edges are removed
        
        params[4] = 1
        params[5] = k
        params[6] = p
    
    return g, params

### Generating Training Data

In [ ]:
dataArgs = {"n_max": 24, "n_lower": 1, "n_upper": 24, "iter_n": True, "fix_n": False, "diag_offset": 0, "diag_value": 1, "clip": True}
                                                                        # none, exact_n, approx_n             #"diag_offset" - 1, 0, 1
trainArgs = {"input_networks": ["ER", "BA", "SW"], "n_graphs": 1000, "n_params": 7}    
    
    
def generate_data(dataArgs): 
    
    
    ## Data ________________________________

    G = np.zeros((trainArgs["n_graphs"], *calculate_A_shape(dataArgs["n_max"], diag_offset = dataArgs["diag_offset"])))
    Params_arr = np.zeros((trainArgs["n_graphs"], trainArgs["n_params"]))
    Params = list()

    ## Generate Graph Data_______________________________

    for i in tqdm(range(0,trainArgs["n_graphs"])):

        g, params = get_graph(dataArgs, trainArgs)

        g, a = sort_adjacency(g)
        a = pad_matrix(a, dataArgs["n_max"], dataArgs["diag_value"])  # pad adjacency matrix to allow less nodes than n_max and fill diagonal
        a_transformed = reshape_A(a, diag_offset = dataArgs["diag_offset"])        

    
        ## Build Data Arrays___________________________________________________

        G[i] = a_transformed
        Params_arr[i] = params
        
        param_dict = dict()
        param_dict["ER"] = params[0]
        param_dict["p1"] = params[1]
        param_dict["BA"] = params[2]
        param_dict["e"] = params[3]
        param_dict["SW"] = params[4]
        param_dict["k"] = params[5]
        param_dict["p2"] = params[6]

        Params.append(param_dict)
    

    ## Input and Output Size ___________________________________________________________

    Params, input_shape, output_shape = prepare_in_out(Params, dataArgs["diag_offset"], calculate_A_shape(dataArgs["n_max"], dataArgs["diag_offset"]))
    print("input_shape:", input_shape, ", output_shape:", output_shape)
    
    ## scale parameters in T_array for smoother training
    scaler = MinMaxScaler()
    scaler.fit(Params_arr)
    Params_arr = scaler.transform(Params_arr)
    
    return G,Params,Params_arr,input_shape,output_shape,scaler
    
G, Params, Params_arr, input_shape, output_shape, scaler = generate_data(dataArgs)

# beta-VAE (MLP, Conv)

## Build Model

In [ ]:
## libs
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

## Keras
from keras.layers import Lambda, Input, Dense, Conv2D, Conv2DTranspose, Flatten, Reshape
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K
import tensorflow as tf

from sklearn.model_selection import train_test_split


class VAE():

    # reparameterization trick
    # instead of sampling from Q(z|X), sample eps = N(0,I)
    # then z = z_mean + sqrt(var)*eps

    def sampling(self, args):
        """Reparameterization trick by sampling fr an isotropic unit Gaussian.
        # Arguments
            args (tensor): mean and log of variance of Q(z|X)
        # Returns
            z (tensor): sampled latent vector
        """

        z_mean, z_log_var = args
        batch = K.shape(z_mean)[0]
        dim = K.int_shape(z_mean)[1]
        # by default, random_normal has mean=0 and std=1.0
        epsilon = K.random_normal(shape=(batch, dim))
        return z_mean + K.exp(0.5 * z_log_var) * epsilon

    def __init__(self, modelArgs, trainArgs, G, T_array):

        ## MODEL ______________________________________________________________

        ## Multi-layer Perceptron without convolutions__________________________________
        if modelArgs["nn_architecture"] == "mlp":
            ## 1) build encoder model __________________________

            inputs = Input(shape=modelArgs["input_shape"], name='encoder_input')
            x = Dense(128, activation='relu')(inputs)
            x = Dense(64, activation='relu')(x)
            z_mean = Dense(modelArgs["latent_dim"], name='z_mean')(x)
            z_log_var = Dense(modelArgs["latent_dim"], name='z_log_var')(x)

            # use reparameterization trick to push the sampling out as input
            # note that "output_shape" isn't necessary with the TensorFlow backend
            z = Lambda(self.sampling, output_shape=(modelArgs["latent_dim"],), name='z')([z_mean, z_log_var])

            ## 2) build decoder model __________________________

            latent_inputs = Input(shape=(modelArgs["latent_dim"],), name='z_sampling')
            y = Dense(64, activation='relu')(latent_inputs)
            y = Dense(128, activation='relu')(y)
            graph_outputs = Dense(modelArgs["output_shape"], activation='sigmoid')(y)

        ## Convolutional Neural Network_________________________________

        if modelArgs["nn_architecture"] == "2D_conv":

            ## 1) build encoder model____________________________________

            inputs = Input(shape=modelArgs["input_shape"], name='encoder_input')
            x = inputs

            for i in range(2):
                modelArgs['filters'] *= 2
                x = Conv2D(filters=modelArgs['filters'], kernel_size=modelArgs['kernel_size'], activation='relu',
                           strides=2, padding='same')(x)

            # shape info needed to build decoder model
            shape = K.int_shape(x)

            # generate latent vector Q(z|X)
            x = Flatten()(x)
            x = Dense(16, activation='relu')(x)
            z_mean = Dense(modelArgs["latent_dim"], name='z_mean')(x)
            z_log_var = Dense(modelArgs["latent_dim"], name='z_log_var')(x)

            # use reparameterization trick to push the sampling out as input
            # note that "output_shape" isn't necessary with the TensorFlow backend
            z = Lambda(self.sampling, output_shape=(modelArgs["output_shape"],), name='z')([z_mean, z_log_var])

            ## 2) build decoder model____________________________________

            latent_inputs = Input(shape=(modelArgs["latent_dim"],), name='z_sampling')
            x = Dense(shape[1] * shape[2] * shape[3], activation='relu')(latent_inputs)
            x = Reshape((shape[1], shape[2], shape[3]))(x)

            for i in range(2):
                x = Conv2DTranspose(filters=modelArgs['filters'], kernel_size=modelArgs['kernel_size'],
                                    activation='relu', strides=2, padding='same')(x)
                modelArgs['filters'] //= 2

            graph_outputs = Conv2DTranspose(filters=1, kernel_size=modelArgs['kernel_size'], activation='sigmoid',
                                            padding='same', name='decoder_output')(x)

        ## 2.2) decode growth parameters_________________________________

        # g = Dense(4, activation='relu')(latent_inputs)
        g = Dense(4, activation='relu')(latent_inputs)
        g = Dense(6, activation='relu')(g)
        param_outputs = Dense(modelArgs["growth_param"], activation='linear')(g)


        ## 1) instantiate encoder model
        encoder = Model(inputs, [z_mean, z_log_var, z], name='encoder')
        encoder.summary()
        # plot_model(encoder, to_file='vae_mlp_encoder.png', show_shapes=True)

        ## 2) instantiate decoder model
        graph_decoder = Model(latent_inputs, graph_outputs, name='graph_decoder')
        graph_decoder.summary()
        # plot_model(decoder, to_file='vae_mlp_decoder.png', show_shapes=True)

        ## 2.2) decode growth parameters
        param_decoder = Model(latent_inputs, param_outputs, name='param_decoder')
        param_decoder.summary()

        ## 3) instantiate VAE model
        graph_outputs = graph_decoder(encoder(inputs)[2])
        param_outputs = param_decoder(encoder(inputs)[2])
        outputs = [graph_outputs, param_outputs]

        vae = Model(inputs, outputs, name='vae_graph')
        # vae.summary()

        ## TRAINING ______________________________________

        ## Train and Validation Split _______________________________________________

        x_train, x_test, y_train, y_test = train_test_split(G, T_array, test_size=trainArgs["data_split"],
                                                            random_state=1, shuffle=True)

        def reconstr_loss_func(y_true, y_pred):

            ## RECONSTRUCTION LOSS_______________________

            if trainArgs["loss"] == "mse":

                if modelArgs["nn_architecture"] == "mlp":
                    reconstruction_loss = mse(y_true[0], y_pred[0])
                    reconstruction_loss *= modelArgs["input_shape"]

                if modelArgs["nn_architecture"] == "2D_conv":
                    reconstruction_loss = mse(K.flatten(y_true[0]), K.flatten(y_pred[0]))
                    reconstruction_loss *= modelArgs["input_shape"][0] * modelArgs["input_shape"][1]

            if trainArgs["loss"] == "binary_crossentropy":

                if modelArgs["nn_architecture"] == "mlp":
                    reconstruction_loss = binary_crossentropy(y_true[0], y_pred[0])
                    reconstruction_loss *= modelArgs["input_shape"]

                if modelArgs["nn_architecture"] == "2D_conv":
                    reconstruction_loss = binary_crossentropy(K.flatten(y_true[0]), K.flatten(y_pred[0]))
                    reconstruction_loss *= modelArgs["input_shape"][0] * modelArgs["input_shape"][1]

            ## KL LOSS _____________________________________________

            kl_loss = 1 + z_log_var - K.square(z_mean) - K.exp(z_log_var)
            kl_loss = K.sum(kl_loss, axis=-1)
            kl_loss *= -0.5

            ## COMPLETE LOSS __________________________________________________

            reconstr_loss = K.mean(reconstruction_loss + (trainArgs["beta"] * kl_loss))

            return reconstr_loss

        # vae.add_loss(vae_loss)
        # vae_loss = reconstr_loss_func(inputs, outputs[0])

        ## PARAMETER LOSS_______________________

        def param_loss_func(y_true, y_pred):

            param_loss = mse(y_pred[1], y_true[1])

            return param_loss

        
        vae.compile(optimizer='adam', loss=[reconstr_loss_func, param_loss_func], metrics=['accuracy'])
        vae.summary()

        ## TRAIN______________________________________________

        # Set callback functions to early stop training and save the best model so far
        callbacks = [EarlyStopping(monitor='val_loss', patience=trainArgs["early_stop"]), ModelCheckpoint(
            filepath="models/weights/vae_mlp_mnist_latent_dim_" + str(modelArgs["latent_dim"]) + ".h5",
            save_best_only=True)]

        vae.fit(x_train, [x_train, y_train], epochs=trainArgs["epochs"], batch_size=trainArgs["batch_size"],
                callbacks=callbacks, validation_data=(x_test, [x_test, y_test]))
        vae.save_weights("models/weights/vae_mlp_mnist_latent_dim_" + str(modelArgs["latent_dim"]) + ".h5")
        models = (encoder, graph_decoder, param_decoder)

        data = (x_test, y_test)
        
        self.model = (encoder, param_decoder)
        self.data = data

## Train Model

In [ ]:
# network parameters
modelArgs = {"nn_architecture": "mlp", "latent_dim": 2, "growth_param": Params_arr.shape[1], "filters": 16, "kernel_size": 3, "input_shape": input_shape, "output_shape": output_shape}
trainArgs = {"beta": 1, "loss": "binary_crossentropy", "weights": "train", "early_stop": 3, "batch_size": 64, "epochs": 50, "data_split": 0.2}

vae = VAE(modelArgs, trainArgs, G, Params_arr)

model = vae.model 
data = vae.data


## Loading Real-World Graph Data

In [ ]:
## load graph data
g_complete = nx.karate_club_graph()

n_complete = len(g_complete)
e_complete = len(g_complete.edges())
a_complete = nx.adjacency_matrix(g_complete)
max_degree = max([d for n, d in g_complete.degree()])

print("number of nodes:", n_complete)
print("number of edges:", e_complete)
print("max_degree:", max_degree)

## Sample Subgraph and Pre-Process

**exact_n:** biased_random_walk, bfs, forestfire, random_walk_induced_graph_sampling, random_walk_sampling_with_fly_back, adjacency, select

**approx_n:**  snowball, standard_bfs, walk, jump

In [ ]:
sampleArgs = {"sample": "biased_random_walk", "jump_bias": "random_walk_induced_graph_sampling", "n": 10, "p": 20.0, "q": 100.0, "source_starts": 2, "source_returns": 4, "depth": 2}

##exact_n: forestfire, random_walk_induced_graph_sampling, random_walk_sampling_with_fly_back, adjacency, select
##approx_n: snowball, bfs, walk, jump

def get_graph(sampleArgs,g_complete,a_complete):
    
    if sampleArgs["sample"] == "biased_random_walk":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.biased_random_walk(sampleArgs["n"], sampleArgs["p"], sampleArgs["q"])

    if sampleArgs["sample"] == "forestfire":
        sampler = ForestFire.ForestFire(g_complete,a_complete)
        g = sampler.forestfire(sampleArgs["n"])

    if sampleArgs["sample"] == "snowball":
        sampler = Snowball.Snowball(g_complete,a_complete)
        g = sampler.snowball(sampleArgs["source_starts"], sampleArgs["source_returns"])

    if sampleArgs["sample"] == "random_walk_induced_graph_sampling":
        sampler = Random_Walk.Random_Walk(g_complete,a_complete)
        g = sampler.random_walk_induced_graph_sampling(sampleArgs["n"])

    if sampleArgs["sample"] == "random_walk_sampling_with_fly_back":
        sampler = Random_Walk.Random_Walk(g_complete,a_complete)
        g = sampler.random_walk_sampling_with_fly_back(sampleArgs["n"], sampleArgs["p"])
        
    if sampleArgs["sample"] == "standard_bfs":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.standard_bfs(sampleArgs["source_starts"], sampleArgs["depth"]) 
        
    if sampleArgs["sample"] == "bfs":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.bfs(sampleArgs["n"]) 
        
    if sampleArgs["sample"] == "walk":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.walk(sampleArgs["source_starts"], sampleArgs["source_returns"], sampleArgs["p"])        
        
    if sampleArgs["sample"] == "jump":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.jump(sampleArgs["source_starts"], sampleArgs["p"], sampleArgs["jump_bias"])
        
    if sampleArgs["sample"] == "adjacency":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.adjacency(sampleArgs["n"]) 
        
    if sampleArgs["sample"] == "select":
        sampler = Base_Samplers.Base_Samplers(g_complete,a_complete)
        g = sampler.adjacency(sampleArgs["n"]) 
    
    return g 

start_time = time.time()
g = get_graph(sampleArgs,g_complete,a_complete)

print("-- n_max should be >=", len(g), "--")
print("-- function get_graph takes %s secs --" % round((time.time() - start_time),  5))

if len(g) <= 200:
    nx.draw(g, node_color = color_map, with_labels = False)
    
g, a = sort_adjacency(g)
a = pad_matrix(a, dataArgs["n_max"], dataArgs["diag_value"])  # pad adjacency matrix to allow less nodes than n_max and fill diagonal
a_transformed = reshape_A(a, diag_offset = dataArgs["diag_offset"]) 
a_transformed = np.reshape(a_transformed, (1,a_transformed.shape[0]))

# Predict Model Parameters

In [ ]:
# ENCODER
encoder, param_decoder = model  ## trained model parts

z_mean, z_log_var, z = encoder.predict(a_transformed, batch_size = trainArgs["batch_size"])


## DECODER

decoded_param = param_decoder.predict(z, batch_size = trainArgs["batch_size"])
decoded_param = np.squeeze(decoded_param)
print(decoded_param)

In [ ]:
fig, ax = plt.subplots()
param_txt = list(Params[0].keys())
param_txt = ["0"] + param_txt
ax.set_xticklabels(param_txt)
x = np.arange(0,7)
rects1 = ax.bar(x, decoded_param, color='b')
plt.show()